In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray

data_dir = Path('../data')

In [2]:
metrics_dir = data_dir / 'outputs/plots/metrics/x1-y1-z1/net_cdf'

def read_plot_metrics(plot_id: str):
    metrics = xr.open_dataset(metrics_dir / f"{plot_id}_with_cover.nc", decode_coords='all')
    metrics.load()
    metrics.close()
    return metrics

In [3]:
plots = gpd.read_file(data_dir / "outputs/plots/plots.geojson")
plots = plots.set_index('id')
plots

,site,plot_number,site_plot_id,geometry
id,,,,
AGG_O_01_P1,AGG_O_01,1,AGG_O_01_P1,"POLYGON ((463042.83 5259846.736, 463025.797 52..."
AGG_O_01_P2,AGG_O_01,2,AGG_O_01_P2,"POLYGON ((463124.556 5259819.234, 463116.068 5..."
AGG_O_01_P3,AGG_O_01,3,AGG_O_01_P3,"POLYGON ((463201.174 5259815.806, 463200.551 5..."
AGG_O_01_P4,AGG_O_01,4,AGG_O_01_P4,"POLYGON ((463257.777 5259801.962, 463245.303 5..."
AGG_O_01_P5,AGG_O_01,5,AGG_O_01_P5,"POLYGON ((463303.022 5259789.552, 463289.794 5..."
...,...,...,...,...
ULY_Y_96_P1,ULY_Y_96,1,ULY_Y_96_P1,"POLYGON ((455348.476 5275834.517, 455348.154 5..."
ULY_Y_96_P2,ULY_Y_96,2,ULY_Y_96_P2,"POLYGON ((455296.576 5275837.224, 455299.223 5..."
ULY_Y_96_P3,ULY_Y_96,3,ULY_Y_96_P3,"POLYGON ((455252.092 5275834.16, 455248.849 52..."


In [4]:
def create_plot_summary(row: gpd.GeoSeries) -> pd.Series:
    id = row.name
    metrics : xr.Dataset = read_plot_metrics(id)

    mean_metrics_names = [
        "point_density",
        "pulse_density",
        "scan_angle_mean",
        "chm",
        "veg_height_mean",
        "veg_height_median",
        "crr",
        "veg_height_q10",
        "veg_height_q20",
        "veg_height_q30",
        "veg_height_q40",
        "veg_height_q50",
        "veg_height_q60",
        "veg_height_q70",
        "veg_height_q80",
        "veg_height_q90",
        "veg_height_sd",
        'veg_height_cv',
        'veg_height_skew',
        'veg_height_kurt',
        'veg_height_gini',
        'canopy_cover_gt1m',
        'fhd',
        'vci',
        'shann_capture',
    ]
    # Skip point and pulse density and scan angle
    sd_metric_names = mean_metrics_names[3:]
    # CV for all the ones that are in height units
    cv_metric_names = [
        "chm",
        "veg_height_mean",
        "veg_height_median",
        "veg_height_q10",
        "veg_height_q20",
        "veg_height_q30",
        "veg_height_q40",
        "veg_height_q50",
        "veg_height_q60",
        "veg_height_q70",
        "veg_height_q80",
        "veg_height_q90",
        "veg_height_sd",
        'fhd',
        'vci',
        'shann_capture'
    ]
    
 

    mean_metrics : pd.Series = metrics[mean_metrics_names].mean(dim=['x', 'y']).to_pandas()
    sd_metrics : pd.Series = metrics[sd_metric_names].std(dim=['x', 'y']).to_pandas()
    cv_metrics : pd.Series = (sd_metrics[cv_metric_names] / mean_metrics[cv_metric_names])
    mean_metrics = mean_metrics.add_prefix('mean__')
    sd_metrics = sd_metrics.add_prefix('sd__')
    cv_metrics = cv_metrics.add_prefix('cv__')

    # I only want max of chm
    max_metrics = pd.Series({
        "max__chm": metrics['chm'].max(dim=['x', 'y']).item()
    })

    # done separately as they may not exist
    cover_metric_names = [
        'groundstorey_capture',
        'understorey_capture',
        'midstorey_capture',
        'upperstorey_capture'
    ]
    cover_metrics = {}

    for cm in cover_metric_names:
        cover_metrics[f'mean__{cm}'] = metrics[cm].mean(dim=['x', 'y']).item() if cm in metrics else 0
        cover_metrics[f'sd__{cm}'] = metrics[cm].std(dim=['x', 'y']).item() if cm in metrics else 0

    cover_metrics = pd.Series(cover_metrics)
    

    plot_summary_metrics = pd.concat([mean_metrics, max_metrics, sd_metrics, cv_metrics, cover_metrics])
    plot_summary_metrics.name = id

    return plot_summary_metrics

In [5]:
create_plot_summary(plots.iloc[0])

mean__point_density          94.731332
mean__pulse_density          48.682927
mean__scan_angle_mean        -0.070185
mean__chm                     9.104318
mean__veg_height_mean         5.137496
                               ...    
sd__understorey_capture       0.144361
mean__midstorey_capture       0.000000
sd__midstorey_capture         0.000000
mean__upperstorey_capture     0.480691
sd__upperstorey_capture       0.215257
Name: AGG_O_01_P1, Length: 72, dtype: float64

In [6]:
plot_summaries = plots.apply(create_plot_summary, axis=1)
plot_summaries

,mean__point_density,mean__pulse_density,mean__scan_angle_mean,mean__chm,mean__veg_height_mean,mean__veg_height_median,mean__crr,mean__veg_height_q10,mean__veg_height_q20,mean__veg_height_q30,...,cv__vci,cv__shann_capture,mean__groundstorey_capture,sd__groundstorey_capture,mean__understorey_capture,sd__understorey_capture,mean__midstorey_capture,sd__midstorey_capture,mean__upperstorey_capture,sd__upperstorey_capture
id,,,,,,,,,,,,,,,,,,,,,
AGG_O_01_P1,94.731332,48.682927,-0.070185,9.104318,5.137496,5.942358,0.555466,0.827366,2.094856,3.608292,...,0.261268,0.255332,0.471075,0.290380,0.158089,0.144361,0.000000,0.000000,0.480691,0.215257
AGG_O_01_P2,94.245378,48.289496,-0.065961,9.143720,5.406901,6.223384,0.587945,1.139656,2.580198,4.114914,...,0.288134,0.289123,0.400177,0.291431,0.169626,0.152461,0.000000,0.000000,0.490357,0.223808
AGG_O_01_P3,96.454386,48.290526,-0.066163,10.808279,6.217011,7.103875,0.566187,1.219969,2.737667,4.482414,...,0.220487,0.214130,0.467292,0.304812,0.210440,0.212543,0.000000,0.000000,0.530290,0.196480
AGG_O_01_P4,109.504584,49.795264,-0.064567,26.233368,13.597761,14.183244,0.515107,2.973598,5.638570,8.552811,...,0.280738,0.268641,0.621307,0.313052,0.616142,0.280960,0.292383,0.294543,0.161525,0.255513
AGG_O_01_P5,121.291362,49.103567,-0.060656,33.688303,18.889891,19.677525,0.559599,6.047861,9.974202,13.305916,...,0.222607,0.215960,0.467454,0.322066,0.550520,0.295910,0.467776,0.308262,0.200391,0.277582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P1,43.126275,27.347843,-11.770692,8.352560,4.404127,4.335475,0.509003,1.691274,2.384248,3.008736,...,0.475823,0.422799,0.625370,0.340075,0.194369,0.237336,0.405743,0.362190,0.010009,0.055076
ULY_Y_96_P2,39.183908,27.518034,-14.802362,6.701582,3.035757,2.953558,0.460762,0.612857,1.049896,1.630950,...,0.620853,0.516626,0.750312,0.270573,0.192010,0.244394,0.193823,0.263624,0.035624,0.123886
ULY_Y_96_P3,37.543771,26.731061,-15.228471,4.673009,1.822269,1.539182,0.441329,0.488729,0.675738,0.897068,...,0.537031,0.450058,0.793417,0.264498,0.281321,0.261548,0.145523,0.207318,0.006595,0.046898


In [8]:
plot_summary_dir = data_dir / 'outputs'
plot_summaries.to_csv(plot_summary_dir / 'plot_summary_metrics.csv')